## Web Application Observability (Ray Serve)

#### Launching a Web Application using Ray Serve

**Imports**

In [ ]:
import logging
import time
import requests
from ray import serve
import json
import numpy as np

💻 Local Environment. Please modify the value of **local_path** to your local path.

Deploy a web application:

In [ ]:
# run the app with default config
!cd scipts/ && serve run main:mnist_app --non-blocking --name app1

Now send http request by running following script. It generates traffic to the Ray Serve web application, allowing you to explore its observability features in the subsequent sections.

In [ ]:
start = time.time()
while time.time() - start < 60:
    images = np.random.rand(2, 1, 28, 28).tolist()
    json_request = json.dumps({"image": images})
    response = requests.post("http://localhost:8000/", json=json_request)
    response.json()["predicted_label"]

#### Ray Serve Metrics

The following metrics are Ray Serve specific:

- **Throughput metrics:**
    - Queries per second (QPS)
    - Error QPS
    - Error by error code QPS

Shown are the throughput metrics for above web application. Click "Metrics" -> "VIEW IN GRAFANA" -> "Dashboards" -> "Serve Dashboard"

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-serve/throughput_per_application.png" alt="Ray Serve Metrics" width="800">

- **Latency metrics:**
    - P50, P90, P99 latencies

Shown are the latency metrics for the MNIST application.

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-serve/latency_per_application.png" alt="Ray Serve Latency Metrics" width="800">

- **Latency and throughput metrics are available at different levels of granularity:**
    - Per-application metrics
    - Per-deployment metrics
    - Per-replica metrics

Shown are the latency metrics on the deployment level.

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-serve/latency_per_deployment.png" alt="Ray Serve Latency Metrics" width="800">

- **Deployment-specific metrics:**
    - Number of replicas
    - Queue size (TODO - explain which queue)

Shown are the number of replicas and queue size for the MNIST application.

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-serve/replicas_per_deployment.png" alt="Ray Serve Deployment Metrics" width="400">

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-serve/queue_size_per_deploymnet.png" alt="Ray Serve Deployment Metrics" width="400">


For Anyscale users, the following metrics are also available:
- **Rollout-specific metrics:**
    - QPS per version
    - Error QPS per version
    - P90 latency per version
    - Number of replicas per version

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-serve/rollouts_per_application.png" alt="Ray Serve Rollout Metrics" width="800">

For details on how to setup custom dashboards and alerts, refer to this [guide in the Anyscale docs](https://docs.anyscale.com/monitoring/custom-dashboards-and-alerting)

#### Ray Serve Logs

💻 Local Environment

To understand system-level behavior and to surface application-level details during runtime, you can leverage Ray logging.

**Implementation:**
- Uses Python's standard logging module
- Logger name is "ray.serve"

**Log Output Locations:**
- Logs are sent to stderr
- Logs are written to disk at `/tmp/ray/session_latest/logs/serve/`

**Types of Logs Captured:**
- System-level logs (from Serve controller and proxy)
- Access logs
- Custom user logs from deployment replicas

**Development Environment Behavior:**
- Logs are streamed to the driver Ray program
- Driver program can be either:
    - Python script calling serve.run()
    - serve run CLI command

<div class="alert alert-info">

**Note:**
Given Ray Serve uses Python's standard logging module, aggressive logging inside your application will incur a performance penalty. Use logging levels to control the verbosity of your logs and to avoid this penalty when running in production.

</div>

Here is how to use logging in a deployment.

In [ ]:
@serve.deployment()
class SayHelloDefaultLogging:
    async def __call__(self):
        logger = logging.getLogger("ray.serve")
        logger.info("hello world")


serve.run(SayHelloDefaultLogging.bind())

resp = requests.get("http://localhost:8000/")

#### Logging Configuration

💻 Local Environment

Here are the common configurations for logging.

- `enable_access_log`: Access logs are injected by default into Replica and Proxy logs. By default, it is `True`.
- `log_level`: Set the log level. By default, it is `INFO`.
- `encoding`: Set the encoding of the log file. By default, it is `JSON`.

You can set the logging configuration:
- At the deployment level
- At the serve instance level

Both programmatically or via a configuration file.


In [ ]:
@serve.deployment(logging_config={"log_level": "DEBUG"})
class SayHelloDebugLogging:
    async def __call__(self):
        logger = logging.getLogger("ray.serve")
        logger.debug("hello world")


serve.run(
    SayHelloDebugLogging.bind(),
    logging_config={
        "encoding": "JSON",
        "log_level": "INFO",
        "enable_access_log": False,
    },
)

resp = requests.get("http://localhost:8000/")

#### Ray Serve Tracing (Anyscale Only)

To perform end-to-end tracing of requests, you can use the Anyscale Tracing integration.

See the [tracing guide](https://docs.anyscale.com/monitoring/tracing/) for details.

After following [README.md](tracing_example/README.md) to run the example, a single request's tracing logs display the following hierarchical structure:

```
1. proxy_http_request (Root) - Duration: 245ms
   └── 2. proxy_route_to_replica (APIGateway) - Duration: 240ms
       └── 3. replica_handle_request (APIGateway) - Duration: 235ms
           └── 4. proxy_route_to_replica (UserService) - Duration: 180ms
               └── 5. replica_handle_request (UserService) - Duration: 175ms
                   └── 6. proxy_route_to_replica (DatabaseService) - Duration: 110ms
                       └── 7. replica_handle_request (DatabaseService) - Duration: 105ms
```

#### Ray Serve Alerts

**Alert Types:**
Grafana [can alert](https://grafana.com/docs/grafana/v7.5/alerting/) based on:
- Metric values
- Rate of change
- Metric absence

**Notification Options:**
- Supports multiple [notification channels](https://grafana.com/docs/grafana/v7.5/alerting/notifications/#add-a-notification-channel) (Slack, PagerDuty, etc.)
- Email support planned for future
- Configurable through notification channels

**Documentation:** Full setup details available in Grafana's [official documentation](https://grafana.com/docs/grafana/v7.5/alerting/)

#### Anyscale Ray Serve Observability

🚀 **Anyscale Platform**: This view only exists in Anyscale 

When deploying a web application to Anyscale, utilize Anyscale's advanced observability features to debug and manage the service.

The Logs view enables viewing and filtering logs:

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-observability/4-Ray-Serve/ray_serve_log_filter.png" width="80%" loading="lazy">

Since each deployment has a version ID, rolling back to an older version is easy through the **Versions** view. Zero-downtime rollouts can be performed with one click:

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-observability/4-Ray-Serve/ray_serve_versions.png" width="80%" loading="lazy">
